In [1]:
import numpy as np
from scipy import integrate, interpolate
import matplotlib.pyplot as plt

In [2]:
def get_aux_params(u, j, theta, kappa, sigma, rho):
    alpha = - u**2 /2. - 1j*u/2. + 1j*j*u
    beta = kappa - rho*sigma*j - rho*sigma*1j*u
    gamma = sigma**2/2
    d = np.sqrt(beta**2 - 4.*alpha*gamma)
    r_p = (beta + d)/sigma**2
    r_n = (beta - d)/sigma**2
    g = r_n/r_p
    return alpha, beta, gamma, d, r_p, r_n, g

def get_C_D(u, j, tau, theta, kappa, sigma, rho):
    _, _, _, d, _, r_n, g = get_aux_params(u, j, theta, kappa, sigma, rho)
    d_tau = d*tau
    C = kappa*(r_n*tau - 2./sigma**2*np.log((1.-g*np.exp(-d_tau))/(1.-g)))
    D = r_n*(1.-np.exp(-d_tau))/(1.-g*np.exp(-d_tau))
    return C, D

def get_integrand(v0, x, u, j, tau, theta, kappa, sigma, rho):
    C,D = get_C_D(u, j, tau, theta, kappa, sigma, rho)
    return ((np.exp(C*theta + D*v0 + 1j*u*x)) / (1j*u)).real

def P(v0, x, j, tau, theta, kappa, sigma, rho):
    def wrapper(u):
        return get_integrand(v0,x, u, j, tau, theta, kappa, sigma, rho)    
    
    if tau <= 1e-4:
        tau = 1e-4
    float_epsilon = np.finfo(float).eps
    integral,err =  integrate.quad(wrapper,float_epsilon,4000.0)
    return 0.5 + 1./np.pi * integral
    
def get_call_price(v0, spot, K, tau, theta, kappa, sigma, rho):
    # presume zero risk-free
    x= np.log(spot/K)
    P0 = P(v0, x, 0, tau, theta, kappa, sigma, rho)
    P1 = P(v0, x, 1, tau, theta, kappa, sigma, rho)
    price = spot * P1 - K * P0
    return price

def get_CIR_Sample(NoOfPaths,kappa,gamma,vbar,s,t,v_s):
    delta = 4.0 *kappa*vbar/gamma/gamma
    c= 1.0/(4.0*kappa)*gamma*gamma*(1.0-np.exp(-kappa*(t-s)))
    kappaBar = 4.0*kappa*v_s*np.exp(-kappa*(t-s))/(gamma*gamma*(1.0-np.exp(-kappa*(t-s))))
    sample = c* np.random.noncentral_chisquare(delta,kappaBar,NoOfPaths)
    return sample

def getEVBinMethod(S,v,NoOfBins):
    if (NoOfBins != 1):
        mat  = np.transpose(np.array([S,v]))
       
        # Sort all the rows according to the first column

        val = mat[mat[:,0].argsort()]
        
        binSize = int((np.size(S)/NoOfBins))
         
        expectation = np.zeros([np.size(S),2]) # columns of S and v

        for i in range(1,binSize-1):
            sample = val[(i-1)*binSize:i*binSize,1]
            expectation[(i-1)*binSize:i*binSize,0] =val[(i-1)*binSize:i*binSize,0]
            expectation[(i-1)*binSize:i*binSize,1] =np.mean(sample)
        return expectation

In [3]:
# Price surface using Heston model

theta = 0.132328; kappa = 10.980797; sigma = 1.7; rho = -0.351560; v0 = 0.065690
r = 0.0 # always presume zero risk-free interest
#spot, K = 659.37, 758.28
# spot, K = 659.37, 600.0
# spot, K = 1.0, 0.9099595068019473
spot = 1.0; K_vec = np.linspace(0.8,1.2,num=10)
T_max = 3
T_vec = np.linspace(0.25,T_max,int(((T_max)/0.25)))
float_epsilon = np.finfo(float).eps
#tau_vec = np.linspace(float_epsilon,0.1,num=100)
# tau_vec = np.linspace(float_epsilon,1.,num=13)
price_vec = []
for tau in T_vec:
    price_tau=[get_call_price(v0, spot, k, tau, theta, kappa, sigma, rho) for k in K_vec]
    price_vec.append(np.array(price_tau))
price_vec

[array([0.2090285 , 0.17026637, 0.13430672, 0.10204707, 0.07435474,
        0.05184616, 0.03465636, 0.02234352, 0.01402364, 0.00865698]),
 array([0.2242402 , 0.18976999, 0.15812609, 0.12964499, 0.10456   ,
        0.08296941, 0.06482173, 0.04992279, 0.03796346, 0.02856077]),
 array([0.23874645, 0.2069625 , 0.17777479, 0.15131604, 0.12764696,
        0.10675381, 0.08855217, 0.07289646, 0.05959329, 0.04841695]),
 array([0.25213827, 0.22224103, 0.19471996, 0.16961764, 0.14692799,
        0.12659956, 0.10854119, 0.09262929, 0.07871589, 0.06663681]),
 array([0.26452668, 0.23606314, 0.20979086, 0.18570647, 0.16377286,
        0.14392352, 0.1260675 , 0.11009487, 0.09588204, 0.08329682]),
 array([0.2760622 , 0.24874771, 0.22347088, 0.20020263, 0.17889   ,
        0.15946017, 0.14182461, 0.12588315, 0.11152772, 0.09864574]),
 array([0.28687481, 0.26051654, 0.23606705, 0.21348201, 0.19269993,
        0.17364566, 0.15623377, 0.14037158, 0.12596198, 0.11290576]),
 array([0.29706924, 0.27152902, 0.

In [50]:
dk = K_vec[1] - K_vec[0]
dt = T_vec[1] - T_vec[0]
si_LV = []
for (i_t,tau) in enumerate(T_vec):
    lv_k = np.zeros(len(K_vec)) # local vol for each strike 
    for (i_k,k) in enumerate(K_vec):
        # derivatives w.r.t. T
        if i_t == 0: # left boundary
            v_t = (price_vec[i_t+1][i_k]-price_vec[i_t][i_k])/dt
        elif i_t == len(T_vec)-1: # right boundary
            v_t = (price_vec[i_t][i_k]-price_vec[i_t-1][i_k])/dt
        else:
            v_t = (price_vec[i_t+1][i_k]-price_vec[i_t-1][i_k])/(2.0*dt)
        
        # derivatives w.r.t. K
        if i_k == 0: # left boundary
            v_k = (price_vec[i_t][i_k+1]-price_vec[i_t][i_k])/dk
#             v_kk = (price_vec[i_t][i_k+2] - 2.0*price_vec[i_t][i_k+1] + price_vec[i_t][i_k])/(dk**2)
        elif i_k == len(K_vec)-1: # right boundary
            v_k = (price_vec[i_t][i_k]-price_vec[i_t][i_k-1])/dk
#             v_kk = (price_vec[i_t][i_k] - 2.0*price_vec[i_t][i_k-1] + price_vec[i_t][i_k-2])/(dk**2)
        else:
            v_kk = (price_vec[i_t][i_k+1] - 2.0*price_vec[i_t][i_k] + price_vec[i_t][i_k-1])/(dk**2)
            v_k = (price_vec[i_t][i_k+1]-price_vec[i_t][i_k-1])/(2.0*dk)
            lv_k[i_k] = np.sqrt((v_t+r*k*v_k)/(0.5*k**2 * v_kk))

        
#         lv_k[i_k] = np.sqrt((v_t+r*k*v_k)/(0.5*k**2 * v_kk))
    lv_k[0] = lv_k[1]
    lv_k[len(K_vec)-1]= lv_k[len(K_vec)-2]
    si_LV.append(lv_k)

si_LV = np.matrix(si_LV)
si_lv_func = interpolate.interp2d(K_vec,T_vec,si_LV)

In [87]:
# Run simulations to compute bins

np.random.seed(1)
T = T_vec[-1]
num_steps = int(T*4*3*5) # weekly
# num_paths = 7500
num_paths = 15000
num_paths = 30000
res = np.zeros((num_steps, num_paths))
dt = T/num_steps
t_sim = np.linspace(0.0,T,num_steps+1)

correlation = np.matrix([[1, rho],[rho, 1]])
L = np.linalg.cholesky(correlation)
alpha = kappa*theta
beta = kappa
s = np.zeros([num_steps+1, num_paths]) # log(S)
v = np.zeros([num_steps+1, num_paths])
v[0,:] = v0

for i in range(num_steps):
    # noncentral chisquare processes

    # Volatility
    dW_indep = np.random.normal(0.0,1.0,(2,num_paths))
    dW_indep = (dW_indep-dW_indep.mean(axis=1).reshape(-1,1))/dW_indep.std(axis=1).reshape(-1,1) # normalize
    dW = L*dW_indep*np.sqrt(dt) # each row (sA sB vA vB) 
#     dW   = np.random.multivariate_normal([0,0,0,0], p["correlation"], num_paths).transpose()*np.sqrt(dt)
    drift_term = (alpha - beta*v[i,:])*dt
    dWv = dW[0,:] # A or B
    vol_term = sigma*np.multiply(np.sqrt(v[i,:]),dWv)
    vol_term += 0.25*sigma**2.0 * (np.power(dWv,2) - dt) # Milstein term
    v[i+1,:] = np.maximum(0.0,v[i,:] + drift_term + vol_term) # truncation
    
    # Stock price
    drift_term = (r-0.5*v[i,:])*dt
    dWs = dW[1,:] 
    vol_term = np.multiply(np.sqrt(v[i,:]), dWs)
    s[i+1,:] = s[i,:] + drift_term + vol_term
    
S = np.exp(s) # stock price

# Check simulation result by pricing

In [88]:
mc_call = []
for (ind, _) in enumerate(T_vec):
    t_ind = np.where(t_sim == T_vec[ind])[0][0]
    mc_payoff = np.zeros(len(K_vec)) # slv for each k 
    for (i_k,k) in enumerate(K_vec):
        payoff = S[t_ind] -k
        payoff = np.where(payoff < 0.0, 0.0, payoff) # modify payoff
        mc_payoff[i_k] = payoff.mean()
    mc_call.append(mc_payoff)

In [89]:
mc_call

[array([0.20886402, 0.17002051, 0.13394476, 0.10167419, 0.07411852,
        0.05169206, 0.03456149, 0.02225075, 0.01396395, 0.00863023]),
 array([0.22383711, 0.18932536, 0.15772662, 0.12934752, 0.10443355,
        0.08300552, 0.06499472, 0.0501327 , 0.03821301, 0.02880711]),
 array([0.23852321, 0.20659666, 0.17733904, 0.15087537, 0.12722504,
        0.10633148, 0.08816167, 0.07260079, 0.0594346 , 0.04842343]),
 array([0.25180192, 0.22185671, 0.19430194, 0.16920611, 0.1465338 ,
        0.12625261, 0.10827373, 0.09242752, 0.07855688, 0.06650804]),
 array([0.2642342 , 0.23580006, 0.2096057 , 0.18558248, 0.16364609,
        0.14386213, 0.126132  , 0.1102692 , 0.09613153, 0.08360715]),
 array([0.27631987, 0.2490705 , 0.22388839, 0.20063989, 0.17928558,
        0.15978562, 0.14211471, 0.12618706, 0.11184001, 0.09897404]),
 array([0.28699082, 0.26065031, 0.23622292, 0.21372852, 0.19301846,
        0.17403177, 0.15668271, 0.14087001, 0.12651248, 0.11348871]),
 array([0.29771711, 0.27223304, 0.

In [90]:
# NoOfBins = 25
NoOfBins = 100
expectations = []
for (ind, _) in enumerate(T_vec):
    t_ind = np.where(t_sim == T_vec[ind])[0][0]
    f = getEVBinMethod(S[t_ind],v[t_ind],NoOfBins)
    ind = np.where(f[:,1]==0)[0]
    f = np.delete(f, ind, 0)
    func = lambda x: np.interp(x, f[:,0], f[:,1])
    expectations.append(func)

In [91]:
si_slv = [] # list of lambda expressions for SLV
for (i_t,tau) in enumerate(T_vec):
    slv = np.zeros(len(K_vec)) # slv for each k 
    for (i_k,k) in enumerate(K_vec):
        v_expect = expectations[i_t](k)
        slv[i_k] = si_LV[i_t, i_k] / np.sqrt(v_expect)
#         slv[i_k] = 1.0 # test 

#     si_slv.append(lambda x: np.interp(x, K_vec, slv))
    si_slv.append(slv)
    
si_slv = np.matrix(si_slv)
si_slv_func = interpolate.interp2d(K_vec,T_vec,si_slv)
si_slv_func(T,K_vec).reshape(-1)

array([0.95070831, 0.95342118, 0.95613405, 0.95884692, 0.96155979,
       0.96372659, 0.96534733, 0.96696807, 0.96858881, 0.97020954])

# SLV model simulation

In [92]:
# Run simulations to compute bins

np.random.seed(1)
T = T_vec[-1]
num_steps = int(T*4*3*5) # weekly
# num_paths = 7500
num_paths = 15000
res = np.zeros((num_steps, num_paths))
dt = T/num_steps
t_sim = np.linspace(0.0,T,num_steps+1)

correlation = np.matrix([[1, rho],[rho, 1]])
L = np.linalg.cholesky(correlation)
alpha = kappa*theta
beta = kappa
s = np.zeros([num_steps+1, num_paths]) # log(S)
v = np.zeros([num_steps+1, num_paths])
v[0,:] = v0

for i in range(num_steps):
    t = i*dt
    # noncentral chisquare processes

    # Volatility
    dW_indep = np.random.normal(0.0,1.0,(2,num_paths))
    dW_indep = (dW_indep-dW_indep.mean(axis=1).reshape(-1,1))/dW_indep.std(axis=1).reshape(-1,1) # normalize
    dW = L*dW_indep*np.sqrt(dt) # each row (sA sB vA vB) 

    # Vol of vol
    drift_term = (alpha - beta*v[i,:])*dt
    dWv = dW[0,:] # A or B
    vol_term = sigma*np.multiply(np.sqrt(v[i,:]),dWv)
    vol_term += 0.25*sigma**2.0 * (np.power(dWv,2) - dt) # Milstein term
    v[i+1,:] = np.maximum(0.0,v[i,:] + drift_term + vol_term) # truncation
    
    # Stock price
    drift_term = (r-0.5*v[i,:])*dt
    dWs = dW[1,:] 
    S_i = np.exp(s[i,:])
    slv_i = si_slv_func(t,S_i).reshape(-1)
    vol_term = np.multiply(slv_i, np.multiply(np.sqrt(v[i,:]), dWs))
    s[i+1,:] = s[i,:] + drift_term + vol_term
    
S = np.exp(s) # stock price

In [93]:
# Do pricing using the simulation data
slv_sim = []
for (i_t,tau) in enumerate(T_vec):
    t_ind = np.where(t_sim == T_vec[i_t])[0][0]
    slv_payoff = np.zeros(len(K_vec)) # slv for each k 
    for (i_k,k) in enumerate(K_vec):
        payoff = S[t_ind] -k
        payoff = np.where(payoff < 0.0, 0.0, payoff) # modify payoff
        slv_payoff[i_k] = payoff.mean()
    slv_sim.append(slv_payoff)

In [94]:
slv_sim

[array([0.20913621, 0.17033463, 0.13441575, 0.1022705 , 0.07473327,
        0.05238455, 0.03524676, 0.02290261, 0.0144281 , 0.00888484]),
 array([0.22516497, 0.1907375 , 0.15911662, 0.130725  , 0.10575687,
        0.0843323 , 0.06633527, 0.0515171 , 0.03967593, 0.03030108]),
 array([0.24262179, 0.21088047, 0.18163682, 0.15512138, 0.13127918,
        0.1102057 , 0.09182023, 0.07589267, 0.06242289, 0.05101726]),
 array([0.25787671, 0.22789122, 0.20027119, 0.17511044, 0.15229311,
        0.13172971, 0.11336128, 0.09705911, 0.08277395, 0.07035238]),
 array([0.26569721, 0.23711359, 0.21077655, 0.18671614, 0.16484555,
        0.14503777, 0.12703541, 0.11082163, 0.09641781, 0.08364041]),
 array([0.27593104, 0.24855555, 0.22312329, 0.19970035, 0.17823614,
        0.15872025, 0.1410576 , 0.12511866, 0.11070436, 0.09772134]),
 array([0.28611694, 0.25970081, 0.23521378, 0.21251469, 0.19160727,
        0.17244827, 0.15494537, 0.13906242, 0.12463545, 0.11152113]),
 array([0.29541609, 0.26977081, 0.

In [95]:
price_vec

[array([0.2090285 , 0.17026637, 0.13430672, 0.10204707, 0.07435474,
        0.05184616, 0.03465636, 0.02234352, 0.01402364, 0.00865698]),
 array([0.2242402 , 0.18976999, 0.15812609, 0.12964499, 0.10456   ,
        0.08296941, 0.06482173, 0.04992279, 0.03796346, 0.02856077]),
 array([0.23874645, 0.2069625 , 0.17777479, 0.15131604, 0.12764696,
        0.10675381, 0.08855217, 0.07289646, 0.05959329, 0.04841695]),
 array([0.25213827, 0.22224103, 0.19471996, 0.16961764, 0.14692799,
        0.12659956, 0.10854119, 0.09262929, 0.07871589, 0.06663681]),
 array([0.26452668, 0.23606314, 0.20979086, 0.18570647, 0.16377286,
        0.14392352, 0.1260675 , 0.11009487, 0.09588204, 0.08329682]),
 array([0.2760622 , 0.24874771, 0.22347088, 0.20020263, 0.17889   ,
        0.15946017, 0.14182461, 0.12588315, 0.11152772, 0.09864574]),
 array([0.28687481, 0.26051654, 0.23606705, 0.21348201, 0.19269993,
        0.17364566, 0.15623377, 0.14037158, 0.12596198, 0.11290576]),
 array([0.29706924, 0.27152902, 0.